In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from helper_functions import *
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_hdf("../LO_SARD102_word2vec_CWE.h5", key="LO_SARD102_word2vec_CWE")
dataset = dataset.loc[dataset.is_vulnerable == 1].reset_index(drop=True)
dataset = dataset.fillna(0)

X = numpy.stack(dataset.word2vec_avg.values)
y = numpy.array(dataset.drop(['is_vulnerable', 'word2vec_avg'], axis="columns"))

print(f"Labels: {y.shape[1]} distinct CWEs")
dataset.columns = dataset.columns.str.replace(r"CWE-", "")
dataset

In [ ]:
X = dataset[['word2vec_avg']]
X

In [ ]:
cwemap = pd.read_hdf('cwe_by_research_concepts_relations.h5')

def get_cwe_cluster(cwe_id):
    try:
        return int(cwemap.loc[cwemap.child_ID == cwe_id].ID)
    except:
        # could not find parent entry for this CWE -> already one of the root "Research Concepts"
        return cwe_id
    
def create_cluster_labels(row):
    labels = row[0:79]
    cluster_labels = pd.Series(0, index=[284, 435, 664, 682, 691, 693, 697, 703, 707, 710])
    for cwe in pd.to_numeric(pd.Series(labels[labels == 1].index)):
        cluster_labels.at[get_cwe_cluster(cwe)] = 1
    return cluster_labels
        
y = dataset.apply(create_cluster_labels, axis=1)
y

In [ ]:
X.to_hdf('LO_SARD102_CweCluster_word2vec.h5', key='X')
y.to_hdf('LO_SARD102_CweCluster_word2vec.h5', key='y')